# module

In [ ]:
import numpy as np
import hdbscan
from sklearn.datasets import make_blobs
from scipy.cluster.hierarchy import dendrogram
import plotly.graph_objects as go

def _get_leaves(condensed_tree):
    cluster_tree = condensed_tree[condensed_tree['child_size'] > 1]
    print(len(cluster_tree))
    if cluster_tree.shape[0] == 0:
        # Return the only cluster, the root
        return [condensed_tree['parent'].min()]

    root = cluster_tree['parent'].min()
    return _recurse_leaf_dfs(cluster_tree, root)
  
def _recurse_leaf_dfs(cluster_tree, current_node):
  children = cluster_tree[cluster_tree['parent'] == current_node]['child']
  if len(children) == 0:
      return [current_node,]
  else:
      return sum([_recurse_leaf_dfs(cluster_tree, child) for child in children], [])
  
def get_leaves(cluster_tree):
    """
    cluster_tree: (u, v, lambda_val, child_size, parent)
    """
    root = cluster_tree[:, 2].max()
    print(f"root: {root}")
    return recurse_leaf_dfs(cluster_tree, root)
    

def recurse_leaf_dfs(cluster_tree, current_node):
    print(f"Visiting Node: {current_node}")

    child1 = cluster_tree[cluster_tree[:,2] == current_node][:,0]
    child2 = cluster_tree[cluster_tree[:,2] == current_node][:,1]
    print(f"Children of Node {current_node}: Child1 {child1}, Child2 {child2}")

    if len(child1) == 0 and len(child2) == 0:
        
        return [current_node,]
    else:
        return sum([recurse_leaf_dfs(cluster_tree, child) for child in np.concatenate((child1, child2))], [])
    


def get_linkage_matrix_from_hdbscan(condensed_tree):
    linkage_matrix = []
    raw_tree = condensed_tree._raw_tree
    condensed_tree = condensed_tree.to_pandas()
    cluster_tree = condensed_tree[condensed_tree['child_size'] > 1]
    sorted_condensed_tree = cluster_tree.sort_values(by=['lambda_val','parent'], ascending=True)
    print(f"len of sorted condensed tree: {len(sorted_condensed_tree)}")

    for i in range(0, len(sorted_condensed_tree), 2):
    
        # 偶数行（i）と次の奇数行（i+1）をペアとして取得
        if i + 1 < len(sorted_condensed_tree):
            
            row_a = sorted_condensed_tree.iloc[i]
            row_b = sorted_condensed_tree.iloc[i+1]
            
            # **前提チェック**: lambda_valが同じであることを確認
            if row_a['lambda_val'] != row_b['lambda_val']:
                # lambda_valが異なる場合は、次の処理に進む（結合の前提が崩れる）
                raise ValueError(f"Lambda value mismatch at rows {i} and {i+1}: {row_a['lambda_val']} vs {row_b['lambda_val']}")
                
            # Parent IDが同じであることを確認 (同じ結合の結果である可能性が高い)
            if row_a['parent'] != row_b['parent']:
                # Parent IDが異なる場合は、このペアは単一の結合ではない可能性が高い
                raise ValueError(f"Parent ID mismatch at rows {i} and {i+1}: {row_a['parent']} vs {row_b['parent']}")
            
            child_a = row_a['child']
            child_b = row_b['child']
            lam = row_a['lambda_val']
            
            # count (サイズ) は、結合された2つの子ノードのサイズ合計を使うのが論理的だが、
            # HDBSCANは親ノードのサイズをリストで持っているため、ここではそのサイズを使用
            # より正確には、このParent IDを持つ全子ノードのサイズの合計を使うべきだが、
            # 2行の child_size の合計で暫定的に対応
            total_size = row_a['child_size'] + row_b['child_size']
            parent_id = row_a['parent']

            linkage_matrix.append([
                int(child_a), 
                int(child_b), 
                int(parent_id),
                lam, 
                total_size,
        ])   
    print(f"len of linkage matrix: {len(linkage_matrix)}")


    # 葉ノードに0-N-1のIDを振る
    node_id_map = {}
    current_id = 0
    leaves = _get_leaves(raw_tree)
    print(f"Number of leaves: {len(leaves)}")

    for leaf in leaves:
        node_id_map[int(leaf)] = current_id
        current_id += 1

    print(f"Leaf ID Map Size: {len(node_id_map)}")
    print(f"current id: {current_id}")

    # 結合ノードにIDを振る(linkage matrixのparent)
    for row in linkage_matrix.__reversed__():
        parent_id = row[2]
        if parent_id not in node_id_map:
            node_id_map[parent_id] = current_id
            current_id += 1

        else:
            print(f"Duplicate Parent ID found: {parent_id}")
            raise ValueError(f"Node ID {parent_id} already assigned!")
    print(f"Total Node ID Map Size: {len(node_id_map)}")
    print(f"current_id: {current_id}")

    # linkage matrixを書き換え
    max_lambda = max(row[3] for row in linkage_matrix)
    print(f"Max Lambda: {max_lambda}")
    linkage_matrix_mapped = [ 
        [node_id_map[row[0]], node_id_map[row[1]], node_id_map[row[2]], max_lambda - row[3], row[4]] 
        for row in linkage_matrix.__reversed__()
    ]

    return np.array(linkage_matrix_mapped), node_id_map

Y_CUTOFF = 0.5
def get_dendrogram_segments(Z: np.ndarray):
    """
    Linkage Matrixからデンドログラム描画に必要な座標データを取得します。
    """
    # 実際には可視化しないが、座標計算のために呼び出す
    # no_plot=True を指定すると、描画はスキップされる
    dendro_data = dendrogram(Z, no_plot=True)
    
    # 'icoord': 各結合の水平方向の座標 (x座標)
    # 'dcoord': 各結合の垂直方向の座標 (y座標、結合距離)
    # これらは描画するV字/逆U字のセグメントを定義します。
    
    segments = []
    
    # icoord, dcoord は (4, K) の配列で、Kは結合の数 (N-1)
    # 各 i は (x1, x2, x3, x4) の座標リスト
    for icoords, dcoords in zip(dendro_data['icoord'], dendro_data['dcoord']):
        # (x1, y1), (x2, y2), (x3, y3), (x4, y4) がセグメントの頂点
        x1, x2, x3, x4 = icoords
        y1, y2, y3, y4 = dcoords

        if y1 == 0:
            if y4 == 0:
                new_y1 = y2
                new_y4 = y3
                new_x2 = (x2 + x3) / 2
                new_x3 = (x2 + x3) / 2
            else:
                new_y1 = y2
                new_y4 = y4
                new_x2 = (x2 + x3) / 2
                new_x3 = x3
        else:
            if y4 == 0:
                new_y1 = y1
                new_y4 = y3
                new_x2 = x2
                new_x3 = (x2 + x3) / 2
            else:
                new_y1 = y1
                new_y4 = y4
                new_x2 = x2
                new_x3 = x3
        # if y1 == 0:
        #     y1 = y2
        #     new_x2 = (x2 + x3) / 2
        # if y4 == 0:
        #     y4 = y3
        #     new_x3 = (x2 + x3) / 2
        # if y1 !=0 and y4 !=0:
        #     new_x2 = x2
        #     new_x3 = x3
        

        # y1 = max(y1, Y_CUTOFF)
        # y4 = max(y4, Y_CUTOFF)
        # 1. 垂直線 (左の子ノードから結合点まで)
        segments.append([(x1, new_y1), (x2, y2)]) 
        # 2. 水平線 (結合したノード間)
        segments.append([(new_x2, y2), (new_x3, y3)]) 
        # 3. 垂直線 (右の子ノードから結合点まで)
        segments.append([(x4, new_y4), (x3, y3)]) 
    
    # segments は [[[x_start, y_start], [x_end, y_end]], ...] のリストになる
    return segments


def get_dendrogram_segments2(Z: np.ndarray):
    """
    Linkage Matrixからデンドログラム描画に必要な座標データを取得します。
    """
    # 実際には可視化しないが、座標計算のために呼び出す
    # no_plot=True を指定すると、描画はスキップされる
    dendro_data = dendrogram(Z, no_plot=True)
    
    # 'icoord': 各結合の水平方向の座標 (x座標)
    # 'dcoord': 各結合の垂直方向の座標 (y座標、結合距離)
    # これらは描画するV字/逆U字のセグメントを定義します。
    
    segments = []
    
    # icoord, dcoord は (4, K) の配列で、Kは結合の数 (N-1)
    # 各 i は (x1, x2, x3, x4) の座標リスト
    for icoords, dcoords in zip(dendro_data['icoord'], dendro_data['dcoord']):
        # (x1, y1), (x2, y2), (x3, y3), (x4, y4) がセグメントの頂点
        x1, x2, x3, x4 = icoords
        y1, y2, y3, y4 = dcoords

    
        if y1 == 0:
            y1 = y2
            new_x2 = (x2 + x3) / 2
        if y4 == 0:
            y4 = y3
            new_x3 = (x2 + x3) / 2
        if y1 !=0 and y4 !=0:
            new_x2 = x2
            new_x3 = x3
        

        # y1 = max(y1, Y_CUTOFF)
        # y4 = max(y4, Y_CUTOFF)
        # 1. 垂直線 (左の子ノードから結合点まで)
        segments.append([(x1, y1), (x2, y2)]) 
        # 2. 水平線 (結合したノード間)
        segments.append([(new_x2, y2), (new_x3, y3)]) 
        # 3. 垂直線 (右の子ノードから結合点まで)
        segments.append([(x4, y4), (x3, y3)]) 
    
    # segments は [[[x_start, y_start], [x_end, y_end]], ...] のリストになる
    return segments
def plot_dendrogram_plotly(segments, colors=None, scores=None):
    fig = go.Figure()
    
    for i, seg in enumerate(segments):
        # seg は [[x_start, y_start], [x_end, y_end]]
        x_coords = [seg[0][0], seg[1][0]]
        y_coords = [seg[0][1], seg[1][1]]
        
        color = 'blue' if colors is None else colors[i]
        info = "no" if scores is None else f"{scores[i]:.2f}"
        fig.add_trace(go.Scatter(
            x=x_coords, 
            y=y_coords, 
            mode='lines',
            line=dict(color=color, width=1),
            # showlegend=True,
            hoverinfo='text',
            text=f'Segment {i}: ({x_coords[0]:.2f}, {y_coords[0]:.2f}) to ({x_coords[1]:.2f}, {y_coords[1]:.2f}, score={info})'
        ))
    
    fig.update_layout(
        title='Simple Dendrogram Visualization',
        xaxis_title='Observation Index',
        yaxis_title='Distance / Height',
        hovermode='closest',
        
    )
    # fig.show() # 実行環境によっては直接表示
    
    # 葉ノードに名前を付ける場合は、dendrogram_data['leaves']と葉のy=0の座標を計算する必要があります。
    fig.update_layout(height=800, width=1000)
    # fig.update_traces(line =dict(color=colors, width=0.5))
    return fig

def calculate_strahler(Z_matrix: np.ndarray, n_leaves: int) -> np.ndarray:
    """
    Linkage Matrix (Z) に基づいて、各結合ノードのストラー数（Strahler Number）を計算する。
    
    Args:
        Z_matrix: Linkage Matrix (N-1 x 4のNumPy配列)。
        n_leaves: 元の観測値/葉ノードの数。
        
    Returns:
        np.ndarray: 各結合ノード（Zの各行）に対応するストラー数の配列。
    """
    n_merges = Z_matrix.shape[0]
    
    # 葉ノードのストラー数を初期化 (すべての葉ノードは S=1)
    # インデックス: 0から n_leaves - 1
    strahler_map = {i: 1 for i in range(n_leaves)}
    
    # Zの各行に対応するストラー数を格納するリスト
    merge_strahler_numbers = np.zeros(n_merges, dtype=int)
    
    # Z行列をボトムアップ（行 0 から N-2）で処理
    for i in range(n_merges):
        u_idx = int(Z_matrix[i, 0])  # 結合されるノード u
        v_idx = int(Z_matrix[i, 1])  # 結合されるノード v
        new_idx = n_leaves + i       # 新しく生成されるノード
        
        # 子ノードのストラー数を取得
        s_u = strahler_map.get(u_idx, 1)
        s_v = strahler_map.get(v_idx, 1)
        
        # ストラー数計算ロジック（二分木）
        if s_u == s_v:
            # S_u = S_v の場合、新しいノードのストラー数は S_u + 1
            s_new = s_u + 1
        else:
            # S_u != S_v の場合、新しいノードのストラー数は Max(S_u, S_v)
            s_new = max(s_u, s_v)
        
        # 結果を記録し、マップを更新
        merge_strahler_numbers[i] = s_new
        strahler_map[new_idx] = s_new

    return merge_strahler_numbers

def filter_linkage_matrix_by_strahler(Z_matrix: np.ndarray, S_min: int, N_leaves: int) -> np.ndarray:
    """
    Linkage Matrix (Z) にストラー数 (Strahler Number) を計算し、指定された最小ストラー数以上の結合のみを保持する。
    
    Args:
        Z_matrix (np.ndarray): Linkage Matrix (N-1 x 4)。
        S_min (int): フィルタリングのための最小ストラー数。

    Returns:
        np.ndarray: フィルタリングされたLinkage Matrix。
    """
    # 葉ノード数 (N_obs = Z.shape[0] + 1)
    # N_leaves = len(leaves) 

    # 1. ストラー数 (Strahler Number) の計算
    strahler_numbers = calculate_strahler(Z_matrix, N_leaves)

    # 2. Z_matrix の拡張 (ストラー数を5列目に追加)
    # (u, v, distance, count, strahler)
    Z_with_strahler = np.hstack((Z_matrix, strahler_numbers[:, np.newaxis]))

    # 3. フィルタリングの実行 (例: ストラー数 2 以上)
    # 5列目（インデックス4）がフィルタリング基準
    # check
    print(Z_with_strahler[1, :])
    filtered_Z_by_strahler = Z_with_strahler[Z_with_strahler[:, 5] >= S_min]

    # ストラー数の分布
    unique_strahler, counts = np.unique(strahler_numbers, return_counts=True)
    print("\n--- ストラー数の分布 ---")
    for s_num, count in zip(unique_strahler, counts):
        print(f"ストラー数 {int(s_num)}: {count} 本の枝")
    print("\n--- フィルタリング結果 ---")
    print(f"元のZ行列の行数: {Z_matrix.shape[0]}")
    print(f"ストラー数 >= {S_min} の行数: {filtered_Z_by_strahler.shape[0]}")


    # ----------------------------------------------------
    #

    # インデックスのマッピング

    node_id_map = {}
    current_id = 0
    leaves = get_leaves(filtered_Z_by_strahler)
    print(f"Leaves: {len(leaves)}")

    for leaf in leaves:
        node_id_map[int(leaf)] = current_id
        current_id += 1
    print(f"Number of leaves: {len(leaves)}")

    for row in filtered_Z_by_strahler:
        parent_id = row[2]
        if parent_id not in node_id_map:
            node_id_map[parent_id] = current_id
            current_id += 1
    print(f"Total Node ID Map Size: {len(node_id_map)}")
    print(f"current_id: {current_id}")

    linkage_matrix_mapped_strahler = [ 
        [node_id_map[row[0]], node_id_map[row[1]], node_id_map[row[2]], row[3], strahler_numbers[i]] 
        for i, row in enumerate(filtered_Z_by_strahler)
    ]
    return np.array(linkage_matrix_mapped_strahler), node_id_map


import plotly.express as px
from sklearn.preprocessing import normalize
import umap
def run_hdbscan(X, 
                y, 
                min_samples=25, 
                min_cluster_size=5, 
                max_cluster_size=1000,
                n_neighbors=100, 
                min_dist=1e-3, 
                spread=2.0, 
                n_epochs=500):
    X = normalize(X)
    hdbscan_model = hdbscan.HDBSCAN(
        min_samples=min_samples,
        min_cluster_size=min_cluster_size,
        max_cluster_size=max_cluster_size,
        cluster_selection_method='leaf'
    )
    labels = hdbscan_model.fit_predict(X)
    hdbscan_model.condensed_tree_.plot(select_clusters=True, label_clusters=True, log_size=True, max_rectangles_per_icicle=10)
    umap_model = umap.UMAP(
        n_neighbors=n_neighbors,
        min_dist=min_dist,
        spread=spread,
        n_epochs=n_epochs,
        random_state=42
    )
    embeddings = umap_model.fit_transform(X)

    print(f"クラスタ数: {len(np.unique(labels))}")
    print(f"ノイズ割合: {np.mean(labels == -1):.2%}")

    df = {
        'x': embeddings[:, 0],
        'y': embeddings[:, 1],
        'label': labels.astype(str),
        'index': list(range(len(X)))
    }

    fig = px.scatter(
        df,
        x='x',
        y='y',
        color='label',
        title='HDBSCAN + UMAP クラスタリング可視化',
        color_continuous_scale='Viridis',
        opacity=0.7,
        text='index',
        hover_name=y
    )
    fig.update_traces(marker=dict(size=3))
    fig.update_layout(width=800, height=800)
    fig.show()

    # numpyであればyはastype(str)
    if not isinstance(y, list):
        y = y.astype(str)

    # 実際のラベルの可視化
    if len(np.unique(y)) <= 20:
        fig = px.scatter(
            x=embeddings[:, 0],
            y=embeddings[:, 1],
            color=y,
            title='実際のラベルの可視化',
            color_continuous_scale='Viridis',
            opacity=0.7,
            text=df['index']
            
        )
    else:
        fig = px.scatter(
            x=embeddings[:, 0],
            y=embeddings[:, 1],
            title='実際のラベルの可視化',
            opacity=0.7,
            text=df['index'],
            hover_name=y
        )
    fig.update_traces(marker=dict(size=3))
    fig.update_layout(width=800, height=800)
    fig.show()

    

    # ===== ヒストグラム計算 =====
    unique_labels = np.unique(labels)
    counts = np.array([np.sum(labels == l) for l in unique_labels])

    # ===== Plotlyで可視化 =====
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=unique_labels,
        y=counts,
        marker_color='blue',
        name='Cluster Size'
    ))

    fig.update_layout(
        title='HDBSCAN Cluster Size Distribution',
        xaxis_title='Cluster Label',
        yaxis_title='Number of Points',
        template='plotly_white'
    )

    fig.show()

    print("\n===== 各クラスタの上位単語を表示（最初の10個） =====")
    # ===== 各クラスタの上位単語を表示（最初の10個） =====
    
    for c in range(0, len(np.unique(labels))):
        # labels == c のインデックスを取得
        indices = np.where(labels == c)[0][:10]  # 上位10個
        words = [y[i] for i in indices]
        print(f"Cluster: {c}, {words}")

    print("\n===== 各クラスタの上位単語をHDBSCANの確率でソートして表示 =====")
    # ===== 各クラスタをHDBSCANの確率でソートして表示 =====
    for c in range(0, 25):
        indices = np.where(labels == c)[0]
        # ラベル内の確率で降順ソート
        indices_sorted = sorted(indices, key=lambda x: 1 - hdbscan_model.probabilities_[x])
        words = [y[i] for i in indices_sorted[:10]]  # 上位10個
        print(f"Cluster: {c}, {words}")

    return hdbscan_model


In [2]:
def run_experiment(condensed_tree, S_list=[2,3,4,5]):
    linkage_matrix, node_id_map1 = get_linkage_matrix_from_hdbscan(condensed_tree)
    segments = get_dendrogram_segments(linkage_matrix[:, [0,1,3,4]])
    plot_dendrogram_plotly(segments)

    for S_min in S_list:
        print(f"\n--- ストラー数 S_min = {S_min} でフィルタリング ---")
        filtered_linkage_matrix, node_id_map2 = filter_linkage_matrix_by_strahler(linkage_matrix, S_min=S_min, N_leaves=len(_get_leaves(condensed_tree._raw_tree)))
        segments_filtered = get_dendrogram_segments(filtered_linkage_matrix[:, [0,1,3,4]])
        plot_dendrogram_plotly(segments_filtered).show()

In [3]:
%run ../utils-notebook/imports.py

# a

In [4]:
def compute_stability_python(condensed_tree):

    # 1. 最小クラスタとクラスタ数を定義 (Cythonと同じロジック)
    smallest_cluster = condensed_tree['parent'].min()
    num_clusters = condensed_tree['parent'].max() - smallest_cluster + 1
    
    largest_child = max(condensed_tree['child'].max(), smallest_cluster)

    # 2. lambda_birth の計算 (クラスタの誕生時の最小 lambda)
    # condensed_tree を 'child' でソート
    sorted_child_data = np.sort(condensed_tree[['child', 'lambda_val']], axis=0)
    
    # births_arr は、child ID に対応する lambda_birth を保持する
    births_arr = np.nan * np.ones(largest_child + 1, dtype=np.double)
    
    current_child = -1
    min_lambda = 0

    # NumPyの structured array を Pythonループで処理 (Cythonの loopを模倣)
    for row in range(sorted_child_data.shape[0]):
        child = sorted_child_data[row]['child']
        lambda_ = sorted_child_data[row]['lambda_val']

        if child == current_child:
            min_lambda = min(min_lambda, lambda_)
        elif current_child != -1:
            births_arr[current_child] = min_lambda
            current_child = child
            min_lambda = lambda_
        else:
            # Initialize
            current_child = child
            min_lambda = lambda_

    if current_child != -1:
        births_arr[current_child] = min_lambda
        
    births_arr[smallest_cluster] = 0.0 # ルートクラスタの lambda_birth は 0
    
    # 3. Stability スコアの計算
    
    # NumPyのベクトル演算で高速化可能だが、Cythonを模倣しループで計算
    result_arr = np.zeros(num_clusters, dtype=np.double)
    
    parents = condensed_tree['parent']
    sizes = condensed_tree['child_size']
    lambdas = condensed_tree['lambda_val']

    for i in range(condensed_tree.shape[0]):
        parent = parents[i]
        lambda_ = lambdas[i]
        child_size = sizes[i]
        result_index = parent - smallest_cluster
        
        # Stability(C) = Σ (lambda_death - lambda_birth) * size
        # condensed_treeの各行は、parent が child_size のクラスタを 'lambda_' で吸収/結合するステップを示す
        # この lambda_ は、HDBSCANロジックでは lambda_death と見なされる
        
        lambda_birth = births_arr[parent]
        
        # NOTE: HDBSCANのStability定義は複雑なため、ここはHDBSCANの内部ロジックを正確に模倣する必要があります。
        # オリジナルのCythonコードを再現:
        result_arr[result_index] += (lambda_ - lambda_birth) * child_size
        
    # 4. ID とスコアを辞書に変換
    node_ids = np.arange(smallest_cluster, condensed_tree['parent'].max() + 1)
    result_pre_dict = np.vstack((node_ids, result_arr)).T

    # フィルタリングされていないノードを含むため、dictに変換してIDとスコアを対応させる
    # [ID, Score] のペアの配列を辞書に変換
    return dict(zip(result_pre_dict[:, 0].astype(int), result_pre_dict[:, 1]))

In [5]:
import pickle
file_path = "../18_rapids/result/20251030_190647/condensed_tree_object.pkl"
with open(file_path, 'rb') as f:
    condensed_tree = pickle.load(f)

In [6]:
stability_dict = compute_stability_python(condensed_tree._raw_tree)

In [7]:
linkage_matrix, node_id_map1 = get_linkage_matrix_from_hdbscan(condensed_tree)
# dictを逆にする
node_id_map1_reversed = {v: k for k, v in node_id_map1.items()}
scores = [stability_dict[node_id_map1_reversed[int(row[2])]] for row in linkage_matrix]
print(scores[:10])

len of sorted condensed tree: 344
len of linkage matrix: 172
344
Number of leaves: 173
Leaf ID Map Size: 173
current id: 173
Total Node ID Map Size: 345
current_id: 345
Max Lambda: 2.855132579803467
[31.910241842269897, 30.347901105880737, 35.436582922935486, 15.639745950698853, 14.921130895614624, 1.620884895324707, 3.0410799980163574, 23.363758087158203, 0.5262100696563721, 1.0080173015594482]


In [37]:
denoised_stability = [score for score in stability_dict.values() if score < 600]
fig = px.histogram(
    x=denoised_stability,
    nbins=100,
    title='Stability Score Distribution',
    labels={'x': 'Stability Score', 'y': 'Count'},
    template='plotly_white'
).show()

In [34]:
# color

max_score = 150
min_score = 0
from plotly.colors import sequential

COLOR_SCALE = sequential.Hot # https://plotly.com/python/builtin-colorscales/
def map_score_to_color(score, min_score, max_score, color_scale):
    # スコアを0-1に正規化
    normalized = (min(max_score, score) - min_score) / (max_score - min_score)
    # カラースケールのインデックスを計算
    index = int(normalized * (len(color_scale) - 1))
    return color_scale[index]

colors = [map_score_to_color(score, min_score, max_score, COLOR_SCALE) for score in scores]
colors = sum([[color]*3 for color in colors], [])
scores = sum([[score]*3 for score in scores], [])

In [41]:
segments = get_dendrogram_segments(linkage_matrix[:, [0,1,3,4]])
fig = plot_dendrogram_plotly(segments, colors=colors, scores=scores)
fig.update_layout(height=2000)
fig.show()